https://huggingface.co/blog/patchtsmixer

# Module Import

In [6]:
# Standard
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import random

# Third Party
from transformers import (
    EarlyStoppingCallback,
    PatchTSMixerConfig,
    PatchTSMixerForPrediction,
    Trainer,
    TrainingArguments,
)
import numpy as np
import pandas as pd
import torch

# First Party
from tsfm_public.toolkit.dataset import ForecastDFDataset
from tsfm_public.toolkit.time_series_preprocessor import TimeSeriesPreprocessor
from tsfm_public.toolkit.util import select_by_index

# Set Seed

In [ ]:
from transformers import set_seed

set_seed(42)

# Dataset Load and ready

- dataset_path: 로컬 .csv 파일의 경로 또는 관심 있는 데이터의 csv 파일에 대한 웹 주소입니다. 데이터는 판다로 로드되므로 pd.read_csv에서 지원하는 것은 무엇이든 지원됩니다: (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).
- timestamp_column: 타임스탬프 정보가 포함된 열 이름, 해당 열이 없는 경우 None을 사용합니다.
- id_columns: 서로 다른 시계열의 ID를 지정하는 열 이름 목록입니다. ID 열이 없는 경우 []를 사용합니다.
- forecast_columns: 모델링할 열 목록입니다.
- context_length: 모델에 입력으로 사용되는 과거 데이터의 양입니다. 입력 데이터 프레임에서 context_length와 같은 길이를 가진 입력 시계열 데이터의 윈도우가 추출됩니다. 다중 시계열 데이터 집합의 경우, 컨텍스트 윈도우는 단일 시계열(즉, 단일 ID) 내에 포함되도록 만들어집니다.
- forecast_horizon: 앞으로 예측할 타임스탬프의 수입니다.
- train_start_index, train_end_index: 학습 데이터를 나타내는 로드된 데이터의 시작 및 종료 인덱스입니다.
- valid_start_index, valid_end_index: 유효성 검사 데이터를 나타내는 로드된 데이터의 시작 및 종료 인덱스입니다.
- test_start_index, test_end_index: 로드된 데이터에서 테스트 데이터를 나타내는 시작 및 종료 인덱스입니다.
- num_workers: PyTorch 데이터로더의 CPU 워커 수입니다.
- batch_size: 배치 크기. 데이터는 먼저 Pandas 데이터 프레임에 로드되고 훈련, 검증 및 테스트 부분으로 나뉩니다. 그런 다음 Pandas 데이터 프레임은 학습에 필요한 적절한 PyTorch 데이터 세트로 변환됩니다.

In [302]:
from glob import glob
# Download ECL data from https://github.com/zhouhaoyi/Informer2020
dataset_path = glob(os.path.join(os.path.abspath(os.path.pardir), "data", "raw", "*.parquet"))
timestamp_column = "STCK_CNTG_HOUR"
id_columns = []

context_length = 512 # 512
forecast_horizon = 96 # 96
num_workers = 16  # Reduce this if you have low number of CPU cores
batch_size = 2  # Adjust according to GPU memory

In [308]:
data = pd.read_parquet(
    dataset_path[11],
    # parse_dates=[timestamp_column],
)
######################
from gluonts.time_feature import get_lags_for_frequency, time_features_from_frequency_str

data["AMOUNT"] = data["STCK_PRPR"] * data["CNTG_VOL"]
data["target"] = data["STCK_PRPR"].pct_change(periods=context_length).fillna(0.)

# freq = "1s"
# lags_sequence = get_lags_for_frequency(freq)
# time_features = time_features_from_frequency_str(freq)

# timestamp_as_index = pd.DatetimeIndex(data[timestamp_column])
# additional_features = [
#     (time_feature.__name__, time_feature(timestamp_as_index))
#     for time_feature in time_features
# ]
# data = pd.concat([data, pd.DataFrame(dict(additional_features))], axis=1)

######################

forecast_columns = list(data.columns.difference([timestamp_column, "MKSC_SHRN_ISCD", "target", "day_of_year"]))
forecast_columns

['AMOUNT', 'CCLD_DVSN', 'CNTG_VOL', 'STCK_PRPR']

In [309]:
# get split
num_train = int(len(data) * 0.7)
num_test = int(len(data) * 0.2)
num_valid = len(data) - num_train - num_test
border1s = [
    0,
    num_train - context_length,
    len(data) - num_test - context_length,
]
border2s = [num_train, num_train + num_valid, len(data)]

In [310]:
print("Border 1s :",border1s)
print("Border 2s :",border2s)

Border 1s : [0, 42636, 48801]
Border 2s : [43148, 49313, 61641]


In [311]:
train_start_index = border1s[0]  # None indicates beginning of dataset
train_end_index = border2s[0]

# Validation의 시작을 컨텍스트 길이만큼 뒤로 이동하여 첫 번째 validation 타임스탬프가 Train 데이터 바로 다음에 오도록 합니다.
valid_start_index = border1s[1]
valid_end_index = border2s[1]

test_start_index = border1s[2]
test_end_index = border2s[2]

In [312]:
print("Train Start Index :", train_start_index)
print("Train End Index :", train_end_index)

print("Validation Start Index :", valid_start_index)
print("Validation End Index :", valid_end_index)

print("Test Start Index :", test_start_index)
print("Test End Index :", test_end_index)

Train Start Index : 0
Train End Index : 43148
Validation Start Index : 42636
Validation End Index : 49313
Test Start Index : 48801
Test End Index : 61641


In [314]:
train_data = select_by_index(
    data,
    id_columns=id_columns,
    start_index=train_start_index,
    end_index=train_end_index,
)
valid_data = select_by_index(
    data,
    id_columns=id_columns,
    start_index=valid_start_index,
    end_index=valid_end_index,
)
test_data = select_by_index(
    data,
    id_columns=id_columns,
    start_index=test_start_index,
    end_index=test_end_index,
)

time_series_processor = TimeSeriesPreprocessor(
    context_length=context_length,
    timestamp_column=timestamp_column,
    id_columns=id_columns,
    input_columns=forecast_columns,
    output_columns=forecast_columns,
    prediction_length=forecast_horizon,
    scaling=False,
    scale_outputs=False
)

time_series_processor.train(train_data)

TimeSeriesPreprocessor {
  "context_length": 512,
  "feature_extractor_type": "TimeSeriesPreprocessor",
  "id_columns": [],
  "input_columns": [
    "AMOUNT",
    "CCLD_DVSN",
    "CNTG_VOL",
    "STCK_PRPR"
  ],
  "output_columns": [
    "AMOUNT",
    "CCLD_DVSN",
    "CNTG_VOL",
    "STCK_PRPR"
  ],
  "prediction_length": 96,
  "processor_class": "TimeSeriesPreprocessor",
  "scale_outputs": false,
  "scaler_dict": {},
  "scaling": false,
  "time_series_task": "forecasting",
  "timestamp_column": "STCK_CNTG_HOUR"
}

In [315]:
train_dataset = ForecastDFDataset(
    time_series_processor.preprocess(train_data),
    id_columns=id_columns,
    timestamp_column="STCK_CNTG_HOUR",
    input_columns=forecast_columns,
    output_columns=forecast_columns,
    context_length=context_length,
    prediction_length=forecast_horizon,
)
valid_dataset = ForecastDFDataset(
    time_series_processor.preprocess(valid_data),
    id_columns=id_columns,
    timestamp_column="STCK_CNTG_HOUR",
    input_columns=forecast_columns,
    output_columns=forecast_columns,
    context_length=context_length,
    prediction_length=forecast_horizon,
)
test_dataset = ForecastDFDataset(
    time_series_processor.preprocess(test_data),
    id_columns=id_columns,
    timestamp_column="STCK_CNTG_HOUR",
    input_columns=forecast_columns,
    output_columns=forecast_columns,
    context_length=context_length,
    prediction_length=forecast_horizon,
    
)

In [316]:
print("past_values :",train_dataset[0]["past_values"].shape)
print("future_values :",train_dataset[0]["future_values"].shape)

past_values : torch.Size([512, 4])
future_values : torch.Size([96, 4])


In [317]:
train_dataset[0]

{'past_values': tensor([[1.5800e+03, 1.0000e+00, 1.0000e+00, 1.5800e+03],
         [7.9000e+04, 1.0000e+00, 5.0000e+01, 1.5800e+03],
         [1.5800e+03, 1.0000e+00, 1.0000e+00, 1.5800e+03],
         ...,
         [1.5900e+03, 1.0000e+00, 1.0000e+00, 1.5900e+03],
         [1.5850e+03, 5.0000e+00, 1.0000e+00, 1.5850e+03],
         [1.5900e+03, 1.0000e+00, 1.0000e+00, 1.5900e+03]]),
 'future_values': tensor([[1.5850e+03, 5.0000e+00, 1.0000e+00, 1.5850e+03],
         [1.5900e+03, 1.0000e+00, 1.0000e+00, 1.5900e+03],
         [1.5850e+03, 5.0000e+00, 1.0000e+00, 1.5850e+03],
         [1.5900e+03, 1.0000e+00, 1.0000e+00, 1.5900e+03],
         [1.5900e+03, 1.0000e+00, 1.0000e+00, 1.5900e+03],
         [1.5850e+03, 5.0000e+00, 1.0000e+00, 1.5850e+03],
         [1.6104e+06, 5.0000e+00, 1.0160e+03, 1.5850e+03],
         [1.5900e+03, 1.0000e+00, 1.0000e+00, 1.5900e+03],
         [1.1095e+04, 5.0000e+00, 7.0000e+00, 1.5850e+03],
         [1.5850e+03, 5.0000e+00, 1.0000e+00, 1.5850e+03],
        

# Configure the PatchTSMixer model
- num_input_channels: 시계열 데이터의 입력 채널(또는 차원) 수입니다. 이 값은 예측 열의 수로 자동 설정됩니다.
- context_length: 위에서 설명한 대로 모델에 입력으로 사용된 과거 데이터의 양입니다.
- prediction_length: 위에서 설명한 예측 기간과 동일합니다.
- patch_length: PatchTSMixer 모델의 패치 길이입니다. context_length를 균등하게 나누는 값을 선택하는 것이 좋습니다.
- patch_stride: 컨텍스트 창에서 패치를 추출할 때 사용되는 보폭입니다.
- d_model: 모델의 숨겨진 피처 차원입니다.
- num_layers: 모델 레이어 수입니다.
- dropout: 인코더에서 FC 레이어에 대한 드롭아웃 확률.
- head_dropout: 모델 헤드에 사용되는 드롭아웃 확률.
- mode: 패치TS믹서 작동 모드. "common_channel"/"mix_channel". 공통 채널은 채널 독립 모드에서 작동합니다. 사전 훈련의 경우 "common_channel"을 사용합니다.
- scaling: 윈도우별 표준 스케일링. 권장 값: "std".

In [319]:
patch_length = 8
config = PatchTSMixerConfig(
    context_length=context_length,
    prediction_length=forecast_horizon,
    patch_length=patch_length,
    num_input_channels=len(forecast_columns),
    patch_stride=patch_length,
    d_model=16,
    num_layers=8,
    expansion_factor=2,
    dropout=0.1, # 0.2
    head_dropout=0.1, # 0.2
    mode="mix_channel",
    scaling=None,
)
model = PatchTSMixerForPrediction(config).to("cuda")


# Train Model

In [ ]:
training_args = TrainingArguments(
    output_dir="./checkpoint/patchtsmixer/financial/pretrain/output/",
    overwrite_output_dir=True,
    learning_rate=0.001,
    num_train_epochs=1,  # 빠르게 테스트하려면 1로 설정하세요.
    do_eval=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    dataloader_num_workers=num_workers,
    report_to="tensorboard",
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=3,
    logging_dir="./checkpoint/patchtsmixer/financial/pretrain/logs/",  # 로깅 디렉터리를 지정
    load_best_model_at_end=True,  # 최적 모델 로드
    metric_for_best_model="eval_loss",  # 중단을 모니터링하는 메트릭
    greater_is_better=False,  # For loss
    label_names=["future_values"],
    fp16=True
)

# 조기 종료 콜백 함수
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=10,  # Validation loss가 10번의 epoch 동안 향상되지 않으면 중단
    early_stopping_threshold=0.0001,  # loss 변화량의 최소값
)

# define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    callbacks=[early_stopping_callback],
)

# pretrain
trainer.train()

In [301]:
results = trainer.evaluate(test_dataset)
print("Test result:")
print(results)

100%|██████████| 6141/6141 [00:41<00:00, 149.74it/s]

Test result:
{'eval_loss': nan, 'eval_runtime': 58.676, 'eval_samples_per_second': 209.302, 'eval_steps_per_second': 104.66, 'epoch': 1.0}


In [ ]:
save_dir = "patchtsmixer/financial/model/pretrain/"
os.makedirs(save_dir, exist_ok=True)
trainer.save_model(save_dir)